In [ ]:
import nglview
import numpy
import openmm
import openmm.app
import openmm.unit
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import Quantity, unit
from openff.units.openmm import ensure_quantity
from openmm.app import Topology as OpenMMTopology

from openff.interchange import Interchange

In [ ]:
def solvate_topology(
    topology: Topology,
    method: str = "pdbfixer",
    box_vectors: Quantity = Quantity(5.0 * numpy.ones(3), unit.nanometer),
) -> Topology:
    if method == "pdbfixer":
        openmm_topology, openmm_positions = _solvate_pdbfixer(
            topology.to_openmm(),
            topology.get_positions().to_openmm(),
            box_vectors=box_vectors.to_openmm(),
        )

        unique_molecules: List[Molecule] = [*topology.unique_molecules]
        unique_molecules.append(Molecule.from_mapped_smiles("[H:2][O:1][H:3]"))

        new_topology = Topology.from_openmm(
            openmm_topology, unique_molecules=unique_molecules
        )
        new_topology.set_positions(ensure_quantity(openmm_positions, "openff"))

        return new_topology


def _solvate_pdbfixer(
    topology: OpenMMTopology,
    positions: openmm.unit.Quantity,
    box_vectors: openmm.unit.Quantity,
) -> tuple[OpenMMTopology, openmm.unit.Quantity]:
    """
    Add solvent and ions using PDBFixer.

    https://htmlpreview.github.io/?https://github.com/openmm/pdbfixer/blob/master/Manual.html

    """
    import pdbfixer

    with open("_tmp.pdb", "w") as _file:
        openmm.app.PDBFile.writeFile(topology, positions, _file)

    pdb_object = pdbfixer.PDBFixer("_tmp.pdb")
    pdb_object.addSolvent(
        boxSize=openmm.Vec3(*box_vectors.value_in_unit(openmm.unit.nanometer))
        * openmm.unit.nanometer
    )

    return pdb_object.topology, pdb_object.positions

In [ ]:
sage_ff14sb = ForceField("openff-2.0.0.offxml", "ff14sb_off_impropers_0.0.3.offxml")

In [ ]:
peptide = Molecule.from_polymer_pdb("ace-a5ca5-nme.pdb")

In [ ]:
solvated_topology = solvate_topology(peptide.to_topology())

In [ ]:
solvated_topology.to_file("solvated.pdb")

In [ ]:
view = nglview.show_file("solvated.pdb")
view.clear_representations()
view.add_representation("ball+stick", selection="all")
view

In [ ]:
interchange = Interchange.from_smirnoff(sage_ff14sb, solvated_topology)